In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import scipy as sp
import pandas as pd
# import geopandas as gpd
# import geoplot as gplt
from shapely.geometry import Point
import shapely
import reverse_geocoder as rg

import sklearn as sk
import matplotlib as mpl
import matplotlib.pylab as plt
import matplotlib.font_manager as fm
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set(rc={'figure.figsize':(13.7,10.27)})
sns.set_style("whitegrid")
sns.set_color_codes()

In [2]:
from dask.distributed import Client
import dask.bag as db
import dask.dataframe as dd
import dask.array as da
import dask

from ast import literal_eval
from collections.abc import MutableMapping
from collections import Counter, defaultdict
# import h5py
import io
import os

In [3]:
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection.split import train_test_split
from surprise.model_selection import cross_validate, GridSearchCV
import pandas as pd
import numpy as np
import os, io
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise import SVDpp, SVD, NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise import accuracy
from sklearn.linear_model import Ridge, RidgeCV, SGDRegressor
from sklearn.metrics import mean_squared_error as mse
import math

In [4]:
SVD_ALGO = "SVD"
BLENDER_RIDGE = "RidgeCV"
BLENDER_SGD = "SGDRegressor"

In [5]:
client = Client(n_workers=8)
client

Client Scheduler: tcp://127.0.0.1:40659 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 16 Memory: 33.65 GB


In [6]:
joined_df = dd.read_csv('data/joined_df.csv').compute()
joined_df = joined_df.loc[:, ~joined_df.columns.str.match('Unnamed')]
joined_df.columns

Index(['rating', 'reviewerName', 'categories', 'gPlusPlaceId', 'gPlusUserId',
       'user_lat', 'user_long', 'placeName', 'price', 'address', 'place_lat',
       'place_long'],
      dtype='object')

In [7]:
def places_filter(df, lat_l, lat_h, lon_l, lon_h):
    return df[(lat_l <= df.place_lat ) & (df.place_lat <= lat_h) & (lon_l <= df.place_long) & (df.place_long <= lon_h)]

In [8]:
%%time
usa_df = places_filter(joined_df, 19.50139, 64.85694, -161.75583, -68.01197)
usa_df.shape

CPU times: user 185 ms, sys: 26 ms, total: 211 ms
Wall time: 208 ms


(2285757, 12)

In [9]:
# Create training set
reader = Reader(rating_scale=(1,5))  #invoke reader instance of surprise library
data = Dataset.load_from_df(usa_df[['gPlusUserId','gPlusPlaceId','rating']], reader)

# Starting Stacking for Collaborative filtering RecSys

# Training first layer for Stacking!

In [10]:
def train_first_layer(algorithms, hparams, train_set, test_set):
    trained_algorithms = []
    performances = []
    
    for idx, algorithm in enumerate(algorithms):
        hparam = hparams[idx]
        print("\t\t{} training started: {}".format(idx+1, hparam))
        if algorithm is SVD or algorithms is SVDpp: configured_algorithm = algorithm(n_factors=hparam["n_factors"], n_epochs=hparam["n_epochs"], reg_all= hparam["reg_all"])
        elif algorithm is NMF: configured_algorithm = algorithm(n_factors=hparam["n_factors"], n_epochs=hparam["n_epochs"])
        elif algorithm is SlopeOne: configured_algorithm = algorithm()
        elif algorithm is CoClustering: configured_algorithm = algorithm(n_cltr_u=hparam["n_cltr_u"], n_cltr_i=hparam["n_cltr_i"], n_epochs=hparam["n_epochs"], random_state=42)
        configured_algorithm.fit(train_set)
        trained_algorithms.append(configured_algorithm)
        rmse = accuracy.rmse(configured_algorithm.test(test_set), verbose=False)
        print("\t\tRMSE= ", rmse)
        performances.append(rmse)
    
    return trained_algorithms, performances

In [11]:
def predict_first_layer(trained_algorithms, test_set):
    return [
        trained_algorithm.test(test_set)
        for trained_algorithm in trained_algorithms
    ]

In [12]:
def train_last_layer(trained_algorithms, train_set, blender_algorithm=BLENDER_RIDGE):
    actual_trainingSet = [each for each in train_set.all_ratings()]
    predictions_for_trainingSet = predict_first_layer(trained_algorithms, actual_trainingSet)
    print("\t\tGenerating predictions Complete !")
    
    train_pred = [[each.est for each in prediction] for prediction in predictions_for_trainingSet]
    train_true = [each.r_ui for each in predictions_for_trainingSet[0]]
    
    blender_train_X = np.column_stack(train_pred)
    blender_train_y = train_true
    print("\t\tOrganizing data for Blender Complete !")
    
    assert blender_train_X.shape[0] == len(blender_train_y), "There's problem in dimension for training set"
    
    blender = None
    if blender_algorithm is BLENDER_RIDGE: blender = RidgeCV(cv=5)
    elif blender_algorithm is BLENDER_SGD: blender = SGDRegressor(max_iter=5000)
    print("\t\tDetermination for Blender Algorithm Complete !")    
    
    blender.fit(blender_train_X, blender_train_y)
    print("\t\tBlender Training Complete !")
    return blender

In [13]:
def predict_last_layer(trained_algorithms, blender, userID, iid):
    preds = [algo.predict(userID, iid) for algo in trained_algorithms]
    blender_X = np.column_stack((pred.est for pred in preds))
    return blender.predict(blender_X)

In [14]:
def CV(cv, algorithms, hparams):
    individual_performances = []
    sgd_performances = []
    ridge_performances = []
    
    for idx in range(cv):
        print("{} Cross Validation started".format(idx+1))
        trainingSet, testSet = train_test_split(data, test_size=0.2, train_size=None, random_state=42, shuffle=True)
        print("\tData Perparation Completed")
        
        trained_algorithms, performances = train_first_layer(algorithms, hparams, trainingSet, testSet)
        individual_performances.append(performances)
        
        predictions = predict_first_layer(trained_algorithms, testSet)
        test_true = [each.r_ui for each in predictions[0]]
        test_pred = [[each.est for each in prediction] for prediction in predictions]
        blender_test_X = np.column_stack(test_pred)
        blender_test_y = test_true
        print("\tPreparing data for Blender Completed")
        
        SGD_blender = train_last_layer(trained_algorithms, trainingSet, BLENDER_SGD)
        final_pred = SGD_blender.predict(blender_test_X)
        sgd_performance = math.sqrt(mse(final_pred, blender_test_y))
        sgd_performances.append(sgd_performance)
        print("\tSGD Performance: ", sgd_performance)
        
        RIDGE_blender = train_last_layer(trained_algorithms, trainingSet, BLENDER_RIDGE)
        final_pred = RIDGE_blender.predict(blender_test_X)
        ridge_performance = math.sqrt(mse(final_pred, blender_test_y))
        ridge_performances.append(ridge_performance)
        print("\tRidge Performance: ", ridge_performance)
    
    return individual_performances, sgd_performances, ridge_performances

# How different would stacking performances be between consists of bests and non-bests?

In [15]:
# Configuration 1 with best models
algorithms = [SVD, SVDpp, NMF]
hparams = [
    {
        "n_factors": 40,
        "n_epochs": 40,
        "reg_all": 0.02
    },
    {
        "n_factors": 10,
        "n_epochs": 10,
        "reg_all": 0.02
    },
    {
        "n_factors": 15,
        "n_epochs": 15
    }
]
individual_performances1, sgd_performances1, ridge_performances1 = CV(3, algorithms, hparams)

1 Cross Validation started
	Data Perparation Completed
		1 training started: {'n_factors': 40, 'n_epochs': 40, 'reg_all': 0.02}
		RMSE=  1.1026735374805523
		2 training started: {'n_factors': 10, 'n_epochs': 10, 'reg_all': 0.02}
		RMSE=  1.1023912653258774
		3 training started: {'n_factors': 15, 'n_epochs': 15}
		RMSE=  1.233697561225039
	Preparing data for Blender Completed
		Generating predictions Complete !
		Organizing data for Blender Complete !
		Determination for Blender Algorithm Complete !
		Blender Training Complete !
	SGD Performance:  1.1132799879776525
		Generating predictions Complete !
		Organizing data for Blender Complete !
		Determination for Blender Algorithm Complete !
		Blender Training Complete !
	Ridge Performance:  1.1741878518117768
2 Cross Validation started
	Data Perparation Completed
		1 training started: {'n_factors': 40, 'n_epochs': 40, 'reg_all': 0.02}
		RMSE=  1.1024764743286801
		2 training started: {'n_factors': 10, 'n_epochs': 10, 'reg_all': 0.02}
		R

In [16]:
# Configuration 2 with non-bests
algorithms = [SVD, SVDpp, NMF]
hparams = [
    {
        "n_factors": 20,
        "n_epochs": 20,
        "reg_all": 0.02
    },
    {
        "n_factors": 8,
        "n_epochs": 8,
        "reg_all": 0.02
    },
    {
        "n_factors": 10,
        "n_epochs": 10
    }
]
individual_performances2, sgd_performances2, ridge_performances2 = CV(3, algorithms, hparams)

1 Cross Validation started
	Data Perparation Completed
		1 training started: {'n_factors': 20, 'n_epochs': 20, 'reg_all': 0.02}
		RMSE=  1.1040119466509086
		2 training started: {'n_factors': 8, 'n_epochs': 8, 'reg_all': 0.02}
		RMSE=  1.104149490483928
		3 training started: {'n_factors': 10, 'n_epochs': 10}
		RMSE=  1.51948432433737
	Preparing data for Blender Completed
		Generating predictions Complete !
		Organizing data for Blender Complete !
		Determination for Blender Algorithm Complete !
		Blender Training Complete !
	SGD Performance:  1.1400390083455985
		Generating predictions Complete !
		Organizing data for Blender Complete !
		Determination for Blender Algorithm Complete !
		Blender Training Complete !
	Ridge Performance:  1.1741878518117768
2 Cross Validation started
	Data Perparation Completed
		1 training started: {'n_factors': 20, 'n_epochs': 20, 'reg_all': 0.02}
		RMSE=  1.1040135273504446
		2 training started: {'n_factors': 8, 'n_epochs': 8, 'reg_all': 0.02}
		RMSE=  

In [17]:
individual_performances1

[[1.1026735374805523, 1.1023912653258774, 1.233697561225039],
 [1.1024764743286801, 1.102518365328133, 1.2336014149937764],
 [1.102466006408648, 1.1025242417518102, 1.2335703215410239]]

In [18]:
sgd_performances1

[1.1132799879776525, 1.1123343009919797, 1.1129575764317414]

In [19]:
# Configuration 1 = Stacking with bests
np.mean(sgd_performances1), np.std(sgd_performances1)

(1.1128572884671246, 0.0003925338419405887)

In [20]:
individual_performances2

[[1.1040119466509086, 1.104149490483928, 1.51948432433737],
 [1.1040135273504446, 1.1040345748623765, 1.519607946594869],
 [1.1041118205891087, 1.1041463122796584, 1.5196839649077607]]

In [21]:
sgd_performances2

[1.1400390083455985, 1.1565842202070509, 1.216636572508115]

In [22]:
# Configuration 2 = Stacking with "so-so"s
np.mean(sgd_performances2), np.std(sgd_performances2)

(1.171086600353588, 0.03290932979509728)

### 1. Stacking with bests models performs better than the other
### 2. Still underperforming, but performance difference is not huge, which might imply performance gets stable than using one algithm
### 3. Even though there is the worst model in second experiment, using stacking "neutralizes" performance